In [1]:
#Module configuration
#from py2neo import Graph #Database connection
import neomodel
import pandas as pd
import datetime

In [2]:
from neomodel import config

neo4jServerAdress = "172.19.242.116"
neo4jConnectionPort = 7687  #Connection to server bolt
user= "neo4j"
password= "Mypassword%2023"
config.DATABASE_URL='bolt://' +user + ':' + password + '@' + neo4jServerAdress + ':' + str(neo4jConnectionPort)

In [3]:
from neomodel import (config, StructuredNode, StringProperty, IntegerProperty,DateTimeProperty,
                    DateTimeFormatProperty,UniqueIdProperty, RelationshipTo,StructuredRel)

config.DATABASE_URL='bolt://' +user + ':' + password + '@' + neo4jServerAdress + ':' + str(neo4jConnectionPort)


#NIP	J_Parcours	DateDebutActe	DateFinActe	UF	Service	Phase_Parcours	Activite	Dimension_Parcours	Ref_Acte	id_Sequence	Type_Sequence	id_sejour

class FOLLOWED_BY(StructuredRel):
    Nb_Days = IntegerProperty(index=True, default=0)
    #NIP=StringProperty(required=True)

class Acte(StructuredNode):
    Ref_Acte = StringProperty(required=True)
    #J_Parcours=IntegerProperty(required=True)
    Duree = IntegerProperty(required=True)
    #DateDebutActe = DateTimeFormatProperty(required=True)  #Would be better to use DateTimeFormatProperty
    #DateFinActe = DateTimeFormatProperty(required=True)  #Would be better to use DateTimeFormatProperty
    UF = StringProperty(required=True)
    Service = StringProperty(required=True)
    Phase_Parcours =StringProperty(required=True)
    Activite =StringProperty(required=True)
    Dimension_Parcours = StringProperty(required=True)
    Sejour_id = StringProperty(required=True)
    Sequence_id = StringProperty(required=True)
    Sequence_Type = StringProperty(required=True)
    
    Carepath = RelationshipTo('Acte', 'FOLLOWED_BY', model=FOLLOWED_BY)

class Patient(StructuredNode):
    NIP = StringProperty(unique_index=True, required=True)
    J0 = StringProperty(required=True)
    J_Parcours = IntegerProperty(required=False, default=0)
    Carepath = RelationshipTo('Acte', 'BEGIN_CAREPATH_WITH', model=FOLLOWED_BY)



In [4]:
# Define the filepath of the import csvfile
file_path = r'..\01_Datasets\DL_Project_Dataset_Sample_V5.csv'
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, sep=';')
# Display the first few rows of the DataFrame
df


,NIP,J_Parcours,DateDebutActe,DateFinActe,UF,Service,Phase_Parcours,Activite,Dimension_Parcours,Ref_Acte,id_Sequence,Type_Sequence,id_sejour
0,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,8532,Médecine Nucléaire,Diagnostic,TEP,Soins,FTNTP2,12513,INIT_NEW,N1771659
1,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,8532,Médecine Nucléaire,Diagnostic,TEP,Soins,ZZQL016,12513,INIT_NEW,N1771659
2,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,1001,Médecine Nucléaire,Diagnostic,TEP,Soins,FTNTP2,12513,INIT_NEW,N160017801
3,N201900002,37,2019-02-08 00:00:00.000,2019-02-08 00:00:00.000,9050,Radiothérapie,Consultation,Consultations,Soins,CS,12513,INIT_NEW,N1791270
4,N201900002,37,2019-02-08 00:00:00.000,2019-02-08 00:00:00.000,9050,Radiothérapie,Consultation,Consultations,Soins,MCS,12513,INIT_NEW,N1791270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,1609,12809,SUIVI_LT,N2134149
1907,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,514,12809,SUIVI_LT,N2134149
1908,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,9005,12809,SUIVI_LT,N2134149
1909,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,522,12809,SUIVI_LT,N2134149


In [5]:
#Function to extract information from a row
date_format = '%Y-%m-%d %H:%M:%S.%f'

def extract_data(row):
    from datetime import datetime
    acte = {
        #'Jp': row.get('J_Parcours', None),
        #'DDA': datetime.strptime(row.get('DateDebutActe', None), date_format),
        #'DDF': datetime.strptime(row.get('DateFinActe', None), date_format),
        'Ref': row.get('Ref_Acte', None),
        'NIP': row.get('NIP', None),
        'UFname': row.get('UF', None),
        'service': row.get('Service', None),
        'phase_parcours': row.get('Phase_Parcours', None).replace(" ", ""),
        'activite': row.get('Activite', None),
        'dim_parcours': row.get('Dimension_Parcours', None),
        'Sequence_id': row.get('id_Sequence', None),
        'Sequence_Type': row.get('Type_Sequence', None),
        'Sejour_id': row.get('id_sejour', None)
    }

    return acte

In [6]:
#Define a query function
def myquery(query, params=None):
    from neomodel import db
    results, meta = db.cypher_query(query, params, resolve_objects=True)
    
    
    return results, meta

In [7]:
#Define a function to create a new acte 
def create_acte(myActe):
    new_acte=Acte(
                    Ref_Acte = myActe['Ref'],
                    Duree = int((myActe['DDF'] - myActe['DDA']).total_seconds()),
                    J_Parcours = myActe['Jp'],
                    DateDebutActe = myActe['DDA'],
                    DateFinActe = myActe['DDF'] ,
                    UF = myActe['UFname'],
                    Service = myActe['service'],
                    Phase_Parcours = myActe['phase_parcours'],
                    Activite = myActe['activite'],
                    Dimension_Parcours = myActe['dim_parcours'],
                    Sejour_id = myActe['Sejour_id'],
                    Sequence_id = myActe['Sequence_id'],
                    Sequence_Type = myActe['Sequence_Type']
                            
                ).save()
    return new_acte

def create_Startnode(myActe):
    startNode=Patient(
                    NIP = myActe['NIP'],
                    J0 = myActe['DDA']
                    ).save()
    return startNode

In [8]:
#MODELIZATION 5 : 
#NOEUDS : ACTES , DIFFERENTIATED BY PHASE PARCOURS.
#RELATIONS : 
    # START : for the first acte
    # ACTES REALIZED BY DAY

#INITIALISATION   #CLEAR THE DATABASE
query="MATCH (n) DETACH DELETE n"
myquery(query)

Start=True
output=True
Previous_Nodes=[]
Active_Nodes=[]

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    myActe = extract_data(row)
    
    #Create a new acte
    new_acte=create_acte(myActe)

    if Start:
        #Create a start node
        startNode=create_Startnode(myActe)

        #LINK THE START NODE TO THE ACTES
        startNode.Carepath.connect(new_acte)
        Start=False

        #SET THE START NODE AS THE PREVIOUS NODE
        Previous_Nodes=[startNode]

    else:
        if oldActe['NIP']!=myActe['NIP']: #Check if we are dealing with the same patient
            Previous_Nodes=[]
            Active_Nodes=[new_acte]
            
            #Create a start node
            startNode=create_Startnode(myActe)

            #LINK THE START NODE TO THE ACTES
            startNode.Carepath.connect(new_acte)
            #SET THE START NODE AS THE PREVIOUS NODE
            Previous_Nodes=[startNode]

        else:            
            #Check if we have to change the previous node
            if myActe['Jp']!=oldActe['Jp']:
                Previous_Nodes=Active_Nodes
                Active_Nodes=[new_acte]

            else:
                Active_Nodes.append(new_acte)

            #LINK THE ACTE TO THE PREVIOUS ONE
            for node in Previous_Nodes:
                node.Carepath.connect(new_acte, {'Nb_Days': myActe['Jp']-node.J_Parcours})
            
    #Store the acte in a variable oldActe
    oldActe=myActe


In [8]:
jim = Person(name='Jim', age=3).save() # Create
jim.age = 4
jim.save() # Update, (with validation)
jim.refresh() # reload properties from the database
jim.element_id # neo4j internal element id

'4:2bcf6dc4-dff0-4e00-9bca-13dc51426442:1'

In [9]:
# Return all nodes
all_nodes = Person.nodes.all()

# Returns Person by Person.name=='Jim' or raises neomodel.DoesNotExist if no match
jim = Person.nodes.get(name='Jim')

In [ ]:
# Will return None unless "bob" exists
someone = Person.nodes.get_or_none(name='bob')

# Will return the first Person node with the name bob. This raises neomodel.DoesNotExist if there's no match.
someone = Person.nodes.first(name='bob')

# Will return the first Person node with the name bob or None if there's no match
someone = Person.nodes.first_or_none(name='bob')

# Return set of nodes
people = Person.nodes.filter(age__gt=3)

In [ ]:
germany = Country(code='DE').save()
jim.country.connect(germany)
berlin = City(name='Berlin').save()
berlin.country.connect(germany)
jim.city.connect(berlin)

if jim.country.is_connected(germany):
    print("Jim's from Germany")

for p in germany.inhabitant.all():
    print(p.name) # Jim

len(germany.inhabitant) # 1

# Find people called 'Jim' in germany
germany.inhabitant.search(name='Jim')

# Find all the people called in germany except 'Jim'
germany.inhabitant.exclude(name='Jim')

# Remove Jim's country relationship with Germany
jim.country.disconnect(germany)

usa = Country(code='US').save()
jim.country.connect(usa)
jim.country.connect(germany)

# Remove all of Jim's country relationships
jim.country.disconnect_all()

jim.country.connect(usa)
# Replace Jim's country relationship with a new one
jim.country.replace(germany)

In [3]:
#CHECK THE CONSTRAINTS IN THE DATABASE
graph.run("SHOW ALL CONSTRAINTS")

id,name,type,entityType,labelsOrTypes,properties,ownedIndex
6,constraint_255e93b,UNIQUENESS,NODE,['Sejour'],['Name'],constraint_255e93b
10,constraint_8a4a96e8,UNIQUENESS,NODE,['Uf'],['Name'],constraint_8a4a96e8
8,constraint_e05260d3,UNIQUENESS,NODE,['Sequence'],['Name'],constraint_e05260d3


In [4]:
len(graph.run("SHOW ALL CONSTRAINTS").data())

3

In [24]:
graph.run("DROP CONSTRAINT constraint_deebdaf8")


(No data)

In [15]:
#CREATE THE CONSTRAINTS
graph.run("CREATE CONSTRAINT FOR (s:Sejour) REQUIRE s.Name IS UNIQUE")
graph.run("CREATE CONSTRAINT FOR (u:Uf) REQUIRE u.Name IS UNIQUE")
graph.run("CREATE CONSTRAINT FOR (a:Acte) REQUIRE a.Ref IS UNIQUE")
graph.run("CREATE CONSTRAINT FOR (seq:Sequence) REQUIRE seq.Name IS UNIQUE")

(No data)

In [18]:
#MAKE THE DATABASE EMPTY BEFORE IMPORTING DATA
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [96]:
result=graph.run("MATCH (u:Uf {id: '8532'}) RETURN u")

for record in result:
    # Access fields by name
    name = record["u"]
    print(name)
type(name)

(_72:Uf {id: '8532'})


py2neo.data.Node

In [5]:

graph.run("MATCH (n) RETURN n;")

n
(_173:Acte {Ref: 'FTNTP2'})
"(_174:Uf {Activite: 'TEP', Dimension_Parcours: 'Soins', Name: '8532', Phase: 'Diagnostic', Service: 'M\u00e9decine Nucl\u00e9aire'})"
(_175:Acte {Ref: 'ZZQL016'})


In [8]:
#MODELIZATION 1 : 
#NOEUDS : SEQUENCE, SEJOURS, ACTES, UF
#RELATIONS : 
    # SEQ-FOLLOWS(NIP)->SEQ 
    # SEJ-FOLLOWS(NIP)->SEJ  
    # ACTES-FOLLOWS(NIP)-> ACTE
    # ACTES-REALIZED_IN(UF attributes)->UF

#INITIALISATION 
graph.run("MATCH (n) DETACH DELETE n") #CLEAR THE DATABASE
old_row=None
output=False

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    acte, uf, sequence, sejour = extract_data(row)
    

    # Check if a node with the same Ref property already exists
    nbNodeUf = len(graph.run(f"MATCH (u:Uf {{Name: '{uf['name']}'}}) RETURN u").data())
    nbNodeActe = len(graph.run(f"MATCH (a:Acte {{Ref: '{acte['Ref']}'}}) RETURN a").data())
    nbNodeSejour=len(graph.run(f"MATCH (s:Sejour {{Name: '{sejour['id']}'}}) RETURN s").data())
    nbNodeSequence=len(graph.run(f"MATCH (s:Sequence {{Name: '{sequence['id']}'}}) RETURN s").data())

    #CREATION DES NODES ACTES
    if nbNodeActe==0:
        # Create Acte node
        query = f"""
        CREATE (:Acte {{Ref: "{acte['Ref']}"}})
        """
        graph.run(query)
    else:
        if output: print(f"Node Acte with Ref {acte['Ref']} already exists")
    
    #CREATION DES NODES UF
    if nbNodeUf==0:
        # Create Uf node
        query = f"""
        CREATE (:Uf {{Name: "{uf['name']}",Service: "{uf['service']}",Phase: "{uf['phase_parcours']}",Activite:"{uf['activite']}",Dimension_Parcours:"{uf['dim_parcours']}"}})
        """
        graph.run(query)
    else:
        if output: print(f"Node UF with id {uf['name']} already exists")

    # Create relationship between Acte and Uf nodes
    query = f"""
    MATCH (a:Acte {{Ref: "{acte['Ref']}"}})
    MATCH (u:Uf {{Name: "{uf['name']}"}})
    MERGE (a)-[:REALIZED_IN {{For_Patient_ID: "{acte['NIP']}", J_Parcours: "{acte['Jp']}", DDA: "{acte['DDA']}", DDF: "{acte['DDF']}"}}]->(u)
    """
    graph.run(query)

    #CREATION DES NODES SEJOURS
    if nbNodeSejour==0:
        # Create Acte node
        query = f"""
        CREATE (:Sejour {{Name: "{sejour['id']}"}})
        """
        graph.run(query)
    else:
        if output: print(f"Node Sejour with Name {sejour['id']} already exists")

    #CREATION DES NODES SEQUENCES
    if nbNodeSequence==0:
        # Create Acte node
        query = f"""
        CREATE (:Sequence {{Name: "{sequence['id']}"}})
        """
        graph.run(query)
    else:
        if output: print(f"Node Sequence with Name {sequence['id']} already exists")

    # Create relationship between Acte and Sejours
    query = f"""
    MATCH (a:Acte {{Ref: "{acte['Ref']}"}})
    MATCH (s:Sejour {{Name: "{sejour['id']}"}})
    MERGE (s)-[:Composed_with]->(a)
    """
    graph.run(query)

    # Create relationship between Sejours and Sequence
    query = f"""
    MATCH (s:Sejour {{Name: "{sejour['id']}"}})
    MATCH (seq:Sequence {{Name: "{sequence['id']}"}})
    MERGE (seq)-[:Composed_with]->(s)
    """
    graph.run(query)

    #CREATE ORDER RELATIONS IN THE JOURNEY
    if output : print('check old row is not none')
    if output :print(old_row)
    if old_row is not None:
        #Recall the previous activity of the patient and connect to the actual activity:
        old_acte, old_uf, old_sequence, old_sejour = extract_data(old_row)
        if output : print('Try to create relations between actes')

        if old_acte['NIP']==acte['NIP']: #Check if we are dealing with the same patient
            if output :  print('NIP and old NIP are equal')
            #then we connect the acte to te previous one
            query = f"""
            MATCH (a:Acte {{Ref: "{acte['Ref']}"}})
            MATCH (pa:Acte {{Ref: "{old_acte['Ref']}"}})
            MERGE (pa)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}"}}]->(a)
            """
            graph.run(query)

            if old_sejour['id']!=sejour['id']: #Check if we are dealing with a new sejour
                if output :  print('Sejour is a new one')
                #then we connect the sejour to te previous one
                query = f"""
                MATCH (s:Sejour {{Name: "{sejour['id']}"}})
                MATCH (ps:Sejour {{Name: "{old_sejour['id']}"}})
                MERGE (ps)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}"}}]->(s)
                """
                graph.run(query)

                if old_sequence['id']!=sequence['id']: #Check if we are dealing with a new sequence
                    if output :  print('Sequence is a new one')
                    #then we connect the sejour to te previous one
                    query = f"""
                    MATCH (seq:Sequence {{Name: "{sequence['id']}"}})
                    MATCH (pseq:Sequence {{Name: "{old_sequence['id']}"}})
                    MERGE (pseq)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}"}}]->(seq)
                    """
                    graph.run(query)

    #STORE THE ACTUAL ROW TO THE PREVIOUS ROW BEFORE NEXT ITTERATION
    old_row = row

In [7]:
#MODELIZATION 2 : 
#NOEUDS : ACTES only.
#RELATIONS : 
    # ACTES-FOLLOWS(NIP)-> ACTE
    # ACTES-REALIZED_IN(UF attributes)->UF

#INITIALISATION 
graph.run("MATCH (n) DETACH DELETE n") #CLEAR THE DATABASE
old_row=None
output=False

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    acte, uf, sequence, sejour = extract_data(row)

    # Check if a node with the same Ref property already exists
    nbNodeUf = len(graph.run(f"MATCH (u:Uf {{Name: '{uf['name']}'}}) RETURN u").data())
    nbNodeActe = len(graph.run(f"MATCH (a:Acte {{Ref: '{acte['Ref']}'}}) RETURN a").data())
    nbNodeSejour=len(graph.run(f"MATCH (s:Sejour {{Name: '{sejour['id']}'}}) RETURN s").data())
    nbNodeSequence=len(graph.run(f"MATCH (s:Sequence {{Name: '{sequence['id']}'}}) RETURN s").data())

    #CREATION DES NODES ACTES
    if nbNodeActe==0:
        # Create Acte node
        query = f"""
        CREATE (:Acte {{Ref: "{acte['Ref']}"}})
        """
        graph.run(query)
    else:
        if output: print(f"Node Acte with Ref {acte['Ref']} already exists")


    if old_row is not None:
        #Recall the previous activity of the patient and connect to the actual activity:
        old_acte, old_uf, old_sequence, old_sejour = extract_data(old_row)
        if output : print('Try to create relations between actes')

        if old_acte['NIP']==acte['NIP']: #Check if we are dealing with the same patient
            if output :  print('NIP and old NIP are equal')
            #then we connect the acte to te previous one
            query = f"""
            MATCH (a:Acte {{Ref: "{acte['Ref']}"}})
            MATCH (pa:Acte {{Ref: "{old_acte['Ref']}"}})
            MERGE (pa)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}", J_Parcours: "{acte['Jp']}"
                        ,DateDebutActe: "{acte['DDA']}",DateFinActe: "{acte['DDF']}"
                        ,UF: "{uf['name']}",Service: "{uf['service']}"
                        ,Phase: "{uf['phase_parcours']}",Activite: "{uf['activite']}"
                        ,Dim_parcours: "{uf['dim_parcours']}"
                        }}]->(a)
            """
            graph.run(query)
    
    #STORE THE ACTUAL ROW TO THE PREVIOUS ROW BEFORE NEXT ITTERATION
    old_row = row

    uf = {
        'name': row.get('UF', None),
        'service': row.get('Service', None),
        'phase_parcours': row.get('Phase_Parcours', None),
        'activite': row.get('Activite', None),
        'dim_parcours': row.get('Dimension_Parcours', None)
    }

In [6]:
#MODELIZATION 3 : 
#NOEUDS : ACTES , DIFFERENTIATED BY PHASE PARCOURS.
#RELATIONS : 
    # START : for the first acte
    # ACTES REALIZED BY DAY
#NEED TO SUPRESS THE CONSTRAINTS !

#INITIALISATION 
graph.run("MATCH (n) DETACH DELETE n") #CLEAR THE DATABASE
old_row=None
output=True

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    acte, uf, sequence, sejour = extract_data(row)
    

    # Check if a node with the same Ref property already exists
    nbNodeUf = len(graph.run(f"MATCH (u:Uf {{Name: '{uf['name']}'}}) RETURN u").data())
    nbNodeActe = len(graph.run(f"MATCH (a:Acte {{Ref: '{acte['Ref']}'}}) RETURN a").data())
    nbNodeSejour=len(graph.run(f"MATCH (s:Sejour {{Name: '{sejour['id']}'}}) RETURN s").data())
    nbNodeSequence=len(graph.run(f"MATCH (s:Sequence {{Name: '{sequence['id']}'}}) RETURN s").data())

    # Create Acte node
    
    query = f"""
    CREATE (:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}",Name: "{uf['name']}",Service: "{uf['service']}",Phase: "{uf['phase_parcours']}",Activite:"{uf['activite']}",Dimension_Parcours:"{uf['dim_parcours']}"}})
    """
    graph.run(query)

    if old_row is None:
        #CREATE THE START NODE
        query = f"""
            CREATE (:Start {{Ref: "{acte['NIP']}", J0: "{acte['DDA']}" }})
            """
        graph.run(query)
        #LINK THE START NODE TO THE ACTES
        query = f"""
            MATCH (s:Start {{Ref: "{acte['NIP']}"}})
            MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}"}})
            MERGE (s)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}", Delay: "{0}"}}]->(a)"""
        graph.run(query)          
    else:
        #Recall the previous activity of the patient and connect to the actual activity:
        old_acte, old_uf, old_sequence, old_sejour = extract_data(old_row)

        if old_acte['NIP']!=acte['NIP']: #Check if we are dealing with the same patient
            query = f"""
                CREATE (:Start {{Ref: "{acte['NIP']}", J0: "{acte['DDA']}" }})
                """
            graph.run(query)
            #LINK THE START NODE TO THE ACTES
            query = f"""
                MATCH (s:Start {{Ref: "{acte['NIP']}"}})
                MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                MERGE (s)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}", Delay: "{0}"}}]->(a)"""
            graph.run(query)   
        else:
            if old_sejour['id']==sejour['id']:
                #LINK THE PREVIOUS ACTE TO THE ACTIVE ACTES
                query = f"""
                    MATCH (pa:Acte{old_uf['phase_parcours']} {{Ref: "{old_acte['Ref']}",NIP: "{old_acte['NIP']}",JP: "{old_acte['Jp']}"}})
                    MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                    MERGE (pa)-[:Followed_by_in_same_sejour {{Delay: "{acte['Jp'] - old_acte['Jp'] }"}}]->(a)"""
                graph.run(query)   
            else:
                if old_sequence['id']==sequence['id']:
                #LINK THE PREVIOUS ACTE TO THE ACTIVE ACTES
                    query = f"""
                        MATCH (pa:Acte{old_uf['phase_parcours']} {{Ref: "{old_acte['Ref']}",NIP: "{old_acte['NIP']}",JP: "{old_acte['Jp']}"}})
                        MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                        MERGE (pa)-[:Followed_by_in_same_sequence {{Delay: "{acte['Jp'] - old_acte['Jp'] }"}}]->(a)"""
                    graph.run(query)   
                else:
                    #LINK THE PREVIOUS ACTE TO THE ACTIVE ACTES
                    query = f"""
                        MATCH (pa:Acte{old_uf['phase_parcours']} {{Ref: "{old_acte['Ref']}",NIP: "{old_acte['NIP']}",JP: "{old_acte['Jp']}"}})
                        MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                        MERGE (pa)-[:Followed_by_in_next_sequence {{Delay: "{acte['Jp'] - old_acte['Jp'] }"}}]->(a)"""
                    graph.run(query)   


    #STORE THE ACTUAL ROW TO THE PREVIOUS ROW BEFORE NEXT ITTERATION
    old_row = row



In [9]:
#MODELIZATION 4 : 
#MODELIZATION 3 - REDUDENCY OF SIMILAR ACTES AT THE SAME TIME
#NOEUDS : ACTES , DIFFERENTIATED BY PHASE PARCOURS.
#RELATIONS : 
    # START : for the first acte
    # ACTES REALIZED BY DAY
#NEED TO SUPRESS THE CONSTRAINTS !

#INITIALISATION 
graph.run("MATCH (n) DETACH DELETE n") #CLEAR THE DATABASE
old_row=None
output=True

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    acte, uf, sequence, sejour = extract_data(row)
    

    # Check if a node with the same Ref property already exists
    nbNodeUf = len(graph.run(f"MATCH (u:Uf {{Name: '{uf['name']}'}}) RETURN u").data())
    nbNodeActe = len(graph.run(f"MATCH (a:Acte {{Ref: '{acte['Ref']}'}}) RETURN a").data())
    nbNodeSejour=len(graph.run(f"MATCH (s:Sejour {{Name: '{sejour['id']}'}}) RETURN s").data())
    nbNodeSequence=len(graph.run(f"MATCH (s:Sequence {{Name: '{sequence['id']}'}}) RETURN s").data())

    #Check if a similar node exists already:
    nbNodeActe = len(graph.run(f"MATCH (a:Acte{uf['phase_parcours']} {{NIP: '{acte['NIP']}',JP: '{acte['Jp']}',Name: '{uf['name']}',Service: '{uf['service']}'}}) RETURN a").data())
    #print(nbNodeActe)

    # Create Acte node
    if nbNodeActe==0:
        query = f"""
        CREATE (:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}",Name: "{uf['name']}",Service: "{uf['service']}",Phase: "{uf['phase_parcours']}",Activite:"{uf['activite']}",Dimension_Parcours:"{uf['dim_parcours']}", Nb_occ: 1}})
        """
        graph.run(query)
    else: #UPDATE THE NODE IF ANOTHER ONE EXISTS
        query = f"""
            MATCH (a:Acte{uf['phase_parcours']} {{NIP: "{acte['NIP']}",JP: "{acte['Jp']}",Name: "{uf['name']}",Service: "{uf['service']}",Phase: "{uf['phase_parcours']}",Activite:"{uf['activite']}",Dimension_Parcours:"{uf['dim_parcours']}"}})
            SET a.Ref=a.Ref+" | " +'{acte['Ref']}', a.Nb_occ=a.Nb_occ+1 
            RETURN a;"""
        graph.run(query)          

    if old_row is None:
        #CREATE THE START NODE
        query = f"""
            CREATE (:Start {{Ref: "{acte['NIP']}", J0: "{acte['DDA']}" }})
            """
        graph.run(query)
        #LINK THE START NODE TO THE ACTES
        query = f"""
            MATCH (s:Start {{Ref: "{acte['NIP']}"}})
            MATCH (a:Acte{uf['phase_parcours']} {{NIP: "{acte['NIP']}"}})
            MERGE (s)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}", Delay: "{0}"}}]->(a)"""
        graph.run(query)          
    else:
        #Recall the previous activity of the patient and connect to the actual activity:
        old_acte, old_uf, old_sequence, old_sejour = extract_data(old_row)

        if old_acte['NIP']!=acte['NIP']: #Check if we are dealing with the same patient
            query = f"""
                CREATE (:Start {{Ref: "{acte['NIP']}", J0: "{acte['DDA']}" }})
                """
            graph.run(query)
            #LINK THE START NODE TO THE ACTES
            query = f"""
                MATCH (s:Start {{Ref: "{acte['NIP']}"}})
                MATCH (a:Acte{uf['phase_parcours']} {{NIP: '{acte['NIP']}',JP: '{acte['Jp']}',Name: '{uf['name']}',Service: '{uf['service']}'}})
                MERGE (s)-[:Followed_by {{For_Patient_ID: "{acte['NIP']}", Delay: "{0}"}}]->(a)"""
            graph.run(query)   
        else:
            if old_sejour['id']==sejour['id']:
                #LINK THE PREVIOUS ACTE TO THE ACTIVE ACTES
                query = f"""
                    MATCH (pa:Acte{old_uf['phase_parcours']} {{NIP: '{old_acte['NIP']}',JP: '{old_acte['Jp']}',Name: '{old_uf['name']}',Service: '{old_uf['service']}'}})
                    MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                    MERGE (pa)-[:Followed_by_in_same_sejour {{Delay: "{acte['Jp'] - old_acte['Jp'] }"}}]->(a)"""
                graph.run(query)   
            else:
                if old_sequence['id']==sequence['id']:
                #LINK THE PREVIOUS ACTE TO THE ACTIVE ACTES
                    query = f"""
                        MATCH (pa:Acte{old_uf['phase_parcours']} {{NIP: '{old_acte['NIP']}',JP: '{old_acte['Jp']}'}})
                        MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                        MERGE (pa)-[:Followed_by_in_same_sequence {{Delay: "{acte['Jp'] - old_acte['Jp'] }"}}]->(a)"""
                    graph.run(query)   
                else:
                    #LINK THE PREVIOUS ACTE TO THE ACTIVE ACTES
                    query = f"""
                        MATCH (pa:Acte{old_uf['phase_parcours']} {{NIP: '{old_acte['NIP']}',JP: '{old_acte['Jp']}'}})
                        MATCH (a:Acte{uf['phase_parcours']} {{Ref: "{acte['Ref']}",NIP: "{acte['NIP']}",JP: "{acte['Jp']}"}})
                        MERGE (pa)-[:Followed_by_in_next_sequence {{Delay: "{acte['Jp'] - old_acte['Jp'] }"}}]->(a)"""
                    graph.run(query)   


    #STORE THE ACTUAL ROW TO THE PREVIOUS ROW BEFORE NEXT ITTERATION
    old_row = row

In [26]:
result_uf =graph.run("MATCH (u:Uf {Name: '8532'}) RETURN u").data()
len(result_uf)

1